In [1]:
# !python make_dataset.py
# !python train_model.py

In [2]:
from cascade.models import ModelRepo
from cascade.meta import MetricViewer
from cascade import data as cdd

In [3]:
from correctness import correctness
from continuity import continuity
from contrastivity import contrastivity
from coherence import coherence
from covariate_complexity import covariate_complexity

c:\xaib-experiments\venv3.8\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from xaib.explainers.feature_importance.constant_explainer import ConstantExplainer
from xaib.explainers.feature_importance.random_explainer import RandomExplainer
from xaib.explainers.feature_importance.shap_explainer import ShapExplainer
from xaib.explainers.feature_importance.lime_explainer import LimeExplainer

In [5]:
BS = 5

In [6]:
# Overwrite previous run
ModelRepo('repo', overwrite=True)

ModelRepo in repo of 0 lines

In [7]:
train_ds = cdd.Pickler('train_ds').ds()
n_features = train_ds.get_meta()[0]['n_features']

In [8]:
explainers = {
    'const': ConstantExplainer(n_features=n_features, constant=1),
    'random': RandomExplainer(n_features=n_features, shift=-15, magnitude=10),
    'shap': ShapExplainer(train_ds),
    'lime': LimeExplainer(train_ds, labels=(0, 1))
}

In [9]:
# correctness(explainers, batch_size=BS)

In [10]:
# continuity(explainers, batch_size=BS)

In [11]:
# contrastivity(explainers, batch_size=BS)

In [12]:
# coherence(explainers, batch_size=BS)

In [13]:
covariate_complexity(explainers, batch_size=BS)

100%|██████████| 4/4 [00:00<00:00, 3993.62it/s]
Exact explainer: 6it [00:19,  3.84s/it]
100%|██████████| 4/4 [00:05<00:00,  1.28s/it]


In [14]:
repo = ModelRepo('repo')

In [15]:
t = MetricViewer(repo).table

In [16]:
def present_metrics(case):
    return t[t['line'].apply(lambda x: case in x)].dropna(axis=1)

In [17]:
present_metrics('correctness')

,line,num,created_at,saved,covariate_regularity,name


In [18]:
present_metrics('continuity')

,line,num,created_at,saved,covariate_regularity,name


In [19]:
present_metrics('contrastivity')

,line,num,created_at,saved,covariate_regularity,name


In [20]:
present_metrics('coherence')

,line,num,created_at,saved,covariate_regularity,name


In [21]:
present_metrics('covariate_complexity')

,line,num,created_at,saved,covariate_regularity,name
0,repo\covariate_complexity,0,2022-12-05 18:01:16.570645+00:00,2 minutes after,-0.000029,const
1,repo\covariate_complexity,1,2022-12-05 18:01:16.570645+00:00,2 minutes after,7.060003,random
2,repo\covariate_complexity,2,2022-12-05 18:01:16.570645+00:00,6 minutes after,9.598662,shap
3,repo\covariate_complexity,3,2022-12-05 18:01:16.570645+00:00,6 minutes after,8.789646,lime
